# In Class Example
Install proper libraries 

In [1]:
!pip install mlxtend

In [2]:
!pip install dmba


# Importing Libraries 


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import dmba
%matplotlib inline

# In Class Example
Here we are exploring the association rules - looking into using the mlxend to create the support, confidence, lift 
We are observing the faceplate of phones and the combination 
This is used to make recommendations on which combinations should be recommended 

In [4]:
import pandas as pd

# Specify the file name (adjust as needed)
uploaded_file_name = 'Faceplate.csv'
# Load the Excel file into a Pandas DataFrame
excel_data = pd.read_csv(uploaded_file_name)

# Display the first few rows of the loaded data
excel_data.head()

,Transaction,Red,White,Blue,Orange,Green,Yellow
0,1,1,1,0,0,1,0
1,2,0,1,0,1,0,0
2,3,0,1,1,0,0,0
3,4,1,1,0,1,0,0
4,5,1,0,1,0,0,0


In [5]:
fp_df = dmba.load_data('Faceplate.csv')
fp_df.set_index('Transaction', inplace=True)
fp_df

,Red,White,Blue,Orange,Green,Yellow
Transaction,,,,,,
1,1,1,0,0,1,0
2,0,1,0,1,0,0
3,0,1,1,0,0,0
4,1,1,0,1,0,0
5,1,0,1,0,0,0
6,0,1,1,0,0,0
7,1,0,1,0,0,0
8,1,1,1,0,1,0
9,1,1,1,0,0,0


The following: used to call the function apriori which will produce the min. support that you determine for this dataset

In [6]:
from pathlib import Path
import heapq
from collections import defaultdict

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules




itemsets = apriori(fp_df, min_support=0.2, use_colnames=True)

#and convert into rules
rules = association_rules(itemsets,metric='confidence',min_threshold=0.5)
rules.sort_values(by=['lift'],ascending=False).head(5)

print(rules.sort_values(by=['lift'], ascending = False).
      drop(columns=['antecedent support', 'consequent support','conviction'])
      .head(6))

       antecedents   consequents  support  confidence      lift  leverage  \
13    (White, Red)       (Green)      0.2         0.5  2.500000      0.12   
15         (Green)  (White, Red)      0.2         1.0  2.500000      0.12   
4          (Green)         (Red)      0.2         1.0  1.666667      0.08   
12  (White, Green)         (Red)      0.2         1.0  1.666667      0.08   
7         (Orange)       (White)      0.2         1.0  1.428571      0.06   
8          (Green)       (White)      0.2         1.0  1.428571      0.06   

    zhangs_metric  
13          1.000  
15          0.750  
4           0.500  
12          0.500  
7           0.375  
8           0.375  


C:\Users\Jake Murillo\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


# Face Plate Data Frame
This creates all the associate rules in clean format


In [7]:
#filter to get rules with single consequents only

rules[[len(c) == 1 for c in rules.consequents]].sort_values(by = ['lift'],ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
13,"(White, Red)",(Green),0.4,0.2,0.2,0.5,2.500000,0.12,1.6,1.000
4,(Green),(Red),0.2,0.6,0.2,1.0,1.666667,0.08,inf,0.500
12,"(White, Green)",(Red),0.2,0.6,0.2,1.0,1.666667,0.08,inf,0.500
7,(Orange),(White),0.2,0.7,0.2,1.0,1.428571,0.06,inf,0.375
8,(Green),(White),0.2,0.7,0.2,1.0,1.428571,0.06,inf,0.375
14,"(Green, Red)",(White),0.2,0.7,0.2,1.0,1.428571,0.06,inf,0.375


Sparse - used to save memory and processing space

In [8]:
#Convert  data set into a spparse data frame
dtype = pd.SparseDtype(int, fill_value=0)

sparse_df = fp_df.astype(dtype)
print("Density {}".format(sparse_df.sparse.density))

#create frequent itemsets
itemsets = apriori(sparse_df, min_support=0.2, use_colnames=True)

#and convert into rules
rules = association_rules(itemsets, metric="confidence" , min_threshold=0.5)
rules.sort_values(by=['lift'], ascending=False).head(6)

Density 0.4000000000000001


C:\Users\Jake Murillo\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
13,"(White, Red)",(Green),0.4,0.2,0.2,0.5,2.500000,0.12,1.6,1.000
15,(Green),"(White, Red)",0.2,0.4,0.2,1.0,2.500000,0.12,inf,0.750
4,(Green),(Red),0.2,0.6,0.2,1.0,1.666667,0.08,inf,0.500
12,"(White, Green)",(Red),0.2,0.6,0.2,1.0,1.666667,0.08,inf,0.500
7,(Orange),(White),0.2,0.7,0.2,1.0,1.428571,0.06,inf,0.375
8,(Green),(White),0.2,0.7,0.2,1.0,1.428571,0.06,inf,0.375


# Inst Cart Example - Homework
Downloading the necessary .csv files into workbook - in this case it is instacart.csv and products.csv

In [9]:
cart = "instacart.csv"
cart = pd.read_csv(cart)
cart.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [10]:
product = 'products.csv'
product = pd.read_csv(product)
product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Here we need to join the two datsets so we use the .merge() function to merge the two datasets that were imported

In [11]:
# Merge the data frames using 'key' column as the common key
merged_df = cart.merge(product, on='product_id', how='left')

merged_df.head()

,order_id,product_id,product_name,aisle_id,department_id
0,1,49302,Bulgarian Yogurt,120,16
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
2,1,10246,Organic Celery Hearts,83,4
3,1,49683,Cucumber Kirby,83,4
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,15


here we store the new dataframe (this is a left join) however we are only storing the necessary columns into this dataframe

In [12]:
left_df = cart.merge(product, how = "left", on=['product_id'])[['order_id','product_name']]

left_df.head()

,order_id,product_name
0,1,Bulgarian Yogurt
1,1,Organic 4% Milk Fat Whole Milk Cottage Cheese
2,1,Organic Celery Hearts
3,1,Cucumber Kirby
4,1,Lightly Smoked Sardines in Olive Oil


we want to put all the product names into the associated with the order_id in column value 

In [13]:
#put products in the same transaction in on line
left_df = left_df.groupby(['order_id'])['product_name'].apply(list).reset_index(name='product_name')

left_df.head(10)

,order_id,product_name
0,1,"[Bulgarian Yogurt, Organic 4% Milk Fat Whole M..."
1,36,"[Grated Pecorino Romano Cheese, Spring Water, ..."
2,38,"[Shelled Pistachios, Organic Biologique Limes,..."
3,96,"[Roasted Turkey, Organic Cucumber, Organic Gra..."
4,98,"[Natural Spring Water, Organic Orange Juice Wi..."
5,112,"[Fresh Cauliflower, I Heart Baby Kale, Sea Sal..."
6,170,"[Asian Chopped Salad with Dressing, XL Emerald..."
7,218,"[Natural Artisan Water, Okra, Organic Yellow P..."
8,226,"[Clementines, Bag, Banana, Kids! Chewables Pro..."
9,349,"[Pure Irish Butter, Pure Almond Unsweetened Or..."


# Sparse Data Frame 

We want to create a list from this dataframe to manipulate the dataframe to create sparse dataframe

The sparse data frame is used to create the 0 and 1 (binary) value storage - In this case we are converting the items into column names and inserting a one if the order item was in the order

In [17]:
#convert dataframe to list, to feed to mlxtend's transactionEncoder

dataset = left_df['product_name'].to_list()

In [18]:
print(dataset)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
dataset[0]

['Bulgarian Yogurt',
 'Organic 4% Milk Fat Whole Milk Cottage Cheese',
 'Organic Celery Hearts',
 'Cucumber Kirby',
 'Lightly Smoked Sardines in Olive Oil',
 'Bag of Organic Bananas',
 'Organic Hass Avocado',
 'Organic Whole String Cheese']

In [20]:
#using TransactionEncoder to do One hot encoding

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary.astype('int')

#This a better way!

#oh_ary = te.fit(datset).transform(dataset, sparse=True)
#sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns =te.columns_)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
te_ary.sum(axis=1)

array([ 8,  8,  9, ...,  8,  7, 28])

In [23]:
# convert the array back to dataframe with the product name as the collumn name. 
fp_df= pd.DataFrame(te_ary, columns=te.columns_)

In [24]:
fp_df.head()

,#2 Coffee Filters,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Greek Yogurt Black Cherry on the Bottom,0% Fat Organic Greek Vanilla Yogurt,0% Fat Peach Greek Yogurt,...,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Density

Simple explaination - how many ones are store in comparsion to the 0 within the table 

(Total Ones in Table/Total Avaiable spots in Table)

In [25]:
# convert the dataframe to sapredataframe.  check density
dtype = pd.SparseDtype(int, fill_value=0)
sparse_df = fp_df.astype(dtype)
print('Density {}'.format(sparse_df.sparse.density))

Density 0.00034618902223081767


# Association Rules
Determining min, support and then running full analysis of the rules 

In [26]:
# create frequent itemsets
itemsets = apriori(sparse_df, min_support=0.01, use_colnames=True)
itemsets

C:\Users\Jake Murillo\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.015985,(100% Whole Wheat Bread)
1,0.012036,(2% Reduced Fat Milk)
2,0.016258,(Apple Honeycrisp Organic)
3,0.029932,(Asparagus)
4,0.116892,(Bag of Organic Bananas)
...,...,...
113,0.010460,"(Organic Avocado, Large Lemon)"
114,0.010754,"(Organic Avocado, Organic Baby Spinach)"
115,0.011763,"(Organic Strawberries, Organic Baby Spinach)"
116,0.011721,"(Organic Strawberries, Organic Hass Avocado)"


In [28]:
# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.1)
rules.sort_values(by=['lift'], ascending=False).head(6)

print(rules.sort_values(by=['lift'], ascending=False)
      .drop(columns=['antecedent support', 'consequent support', 'conviction'])
      .head(6))

               antecedents             consequents   support  confidence  \
20           (Large Lemon)                 (Limes)  0.012183    0.197952   
19                 (Limes)           (Large Lemon)  0.012183    0.266177   
30  (Organic Strawberries)   (Organic Raspberries)  0.011910    0.146134   
29   (Organic Raspberries)  (Organic Strawberries)  0.011910    0.286364   
22           (Large Lemon)       (Organic Avocado)  0.010460    0.169966   
21       (Organic Avocado)           (Large Lemon)  0.010460    0.181686   

        lift  leverage  zhangs_metric  
20  4.324970  0.009366       0.819202  
19  4.324970  0.009366       0.805659  
30  3.513711  0.008520       0.778878  
29  3.513711  0.008520       0.746445  
22  2.952111  0.006917       0.704625  
21  2.952111  0.006917       0.701657  
